In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import openpyxl
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from openpyxl.drawing.image import Image
import win32com.client as win32
import os
from PIL import ImageGrab
import win32com.client

olApp = win32.Dispatch('Outlook.Application')
olNS = olApp.GetNameSpace('MAPI')

import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column
from Defs import concat_columns
from Defs import export_from_WHWEEK

In [2]:
Today ='2024-04-30'
print_to_excel = False

In [3]:
query = f"""
SELECT *
FROM xxwh."unifiedARReport"
WHERE 
    "rowType" = 'M' and
    "reportDate" = TO_DATE('{Today}', 'YYYY-MM-DD') 
    """
# "sourceSystem" in ('SAP ECP', 'SAP ERP') and
data_xxwh = export_from_WHWEEK(query)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:168: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


In [4]:
data_xxwh.columns[data_xxwh.columns.str.contains('hold')]
# data_xxwh[['holding', 'holdingCurrency']]

Index(['holding', 'holdingCurrency'], dtype='object')

In [5]:
# data_xxwh.iloc[:2,150:]
#,partyInternational,customerName,orgName,companyCode,amount,currencyCode,exchangeRate,documentDate,dueDate,presentAmountUSD
data_work = data_xxwh[['companyCode', 'orgName', 'holding', 'functionalCurrency', 'sourceSystem', 'reportDate','documentDate','dueDate','insertToSource','amount','currencyCode','exchangeRate','presentAmountUSD','reportDateUSDRate','debtHoldingCurrency','holdingCurrency','presentationCurrency','reportDateIfrsRateUSDBu','presentAmountIfrsUSD']]
data_work = data_work.fillna(0)
data_USD_1 = data_work[(data_work.currencyCode=='USD')&(data_work.reportDateUSDRate==1)]
data_USD_not1 = data_work[(data_work.currencyCode=='USD')&(data_work.reportDateUSDRate!=1)]
count=0
if print_to_excel == True:
    Output_file=f'{str(date.today())}_USDrate_xxwh-unifiedARReport.xlsx'
    writer = pd.ExcelWriter(Output_file, engine='openpyxl')  
    workbook=writer.book
for USD_data in [data_USD_1, data_USD_not1]:
    count = count+1
    index_list = ['orgName', 'holding',  'reportDate','documentDate','insertToSource','functionalCurrency', 'sourceSystem']
    group_data = USD_data.pivot_table(index=index_list, values=['amount','presentAmountUSD'], aggfunc=sum).reset_index()
    if print_to_excel == True:
        sheet_name=''
        if count == 1:
            sheet_name = 'USD_1'
        if count == 2:
            sheet_name = 'USD_not1'
        group_data.to_excel(writer, sheet_name=sheet_name, index=False)
        col_count=group_data.shape[1]+1
        for col in index_list:
            col_count+=1
            pd.DataFrame({f'Unique_{col}':group_data[col].unique().tolist()}).to_excel(writer, sheet_name=sheet_name, startcol=col_count, index=False)
writer.close()


C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_22912\1057154947.py:15: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  group_data = USD_data.pivot_table(index=index_list, values=['amount','presentAmountUSD'], aggfunc=sum).reset_index()
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_22912\1057154947.py:15: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  group_data = USD_data.pivot_table(index=index_list, values=['amount','presentAmountUSD'], aggfunc=sum).reset_index()


NameError: name 'writer' is not defined

In [15]:
Output_file = f'{str(date.today())}_USDrate_xxwh-unifiedARReport.xlsx'
data_USD_not1[(((data_USD_not1.amount/data_USD_not1.presentAmountUSD).abs() >2)|((data_USD_not1.presentAmountUSD/data_USD_not1.amount).abs() >2))&(data_USD_not1.amount>1)].to_excel(Output_file, sheet_name='USD_not1')
new_list(data_USD_1, Output_file, 'USD_1')